In [1]:
import pickle
import math

from nltk import word_tokenize
from nltk.translate.bleu_score import modified_precision, closest_ref_length, brevity_penalty, SmoothingFunction, sentence_bleu

from collections import Counter

from fractions import Fraction

from modules.sentence import tokenizer, read, detokenize
from modules.model import NMT

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
nbest = 0
weights = (0.25, 0.25, 0.25, 0.25)

In [3]:
from models import fl_en
model = fl_en

In [4]:
source_tokenizer = tokenizer(model.config['source_tokenizer'], lowercase=model.config['source_lowercase'])
source_eval = read('models/fl_en/source.data.test', source_tokenizer, model.config['backwards'])

In [5]:
target_tokenizer = tokenizer('word', lowercase=model.config['target_lowercase'])
references = read('models/fl_en/source.data.test', target_tokenizer, model.config['backwards'])

In [6]:
output_file = open('models/fl_en/result.data.test', 'w', encoding='utf-8')

hypotheses = []

for i, sent in enumerate(model.translate(source_eval, encode=True, nbest=nbest)):
    print(sent, file=output_file, flush=True)
    hypotheses.append(word_tokenize(sent))
    
output_file.close()

In [7]:
evaluation_file = open('models/fl_en/scores.data.test.csv','w', encoding='utf-8')

p_numerators = Counter()
p_denominators = Counter()
hyp_lengths, ref_lengths = 0, 0

for reference, hypothesis in zip(references, hypotheses):
    
    hyp_len = len(hypothesis)
    ref_len = closest_ref_length(references, hyp_len)
    
    hyp_lengths += hyp_len
    ref_lengths += ref_len
    
    set_data = '%d,%d' % (ref_len, hyp_len)
    
    for i, _ in enumerate(weights, start=1):
        p_i = modified_precision(reference, hypothesis, i)
        p_numerators[i] += p_i.numerator
        p_denominators[i] += p_i.denominator
        set_data += ',%d,%d' % (p_i.numerator, p_i.denominator)
    
    print([reference])
    print(hypothesis)
    
    set_data += ',%f' % sentence_bleu([reference], hypothesis)
        
evaluation_file.close()

[['ano', 'ang', 'naging', 'resulta', '?']]
['what', 'does', 'that', 'result', '?']
[['napakalaking', 'pizza', 'niyan', '.']]
['that', 'is', 'a', 'big', 'picta', '.']
[['ito', 'ba', 'ang', 'kauna-unahan', 'mong', 'pagkakataong', 'makapunta', 'ng', 'thailand', '?']]
['is', 'this', 'your', 'first', 'time', 'to', 'call', 'thailand', '?']
[['nasaan', 'ang', 'mga', 'pulseras', '?']]
['where', 'are', 'the', 'bacelts', '?']
[['binasa', 'mo', 'ba', 'ang', 'senyas', 'sa', 'harap', 'mo', '.']]
['did', 'you', 'read', 'the', 'sign', 'in', 'the', 'shop', '.']
[['ano', 'ang', 'nasa', 'isip', 'niyo', ',', 'kung', 'gayon', '?']]
['what', 'do', 'you', 'have', 'in', 'mind', ',', 'then', '?']
[['ito', 'ba', 'ang', 'tamang', 'subway', 'patungo', 'sa', 'london', 'railroad', 'station', '?']]
['is', 'this', 'the', 'right', 'station', 'to', 'london', 'at', 'the', 'london', '?']
[['mayroong', 'nasugatan', '.']]
['somebody', 'is', 'injured', '.']
[['anong', 'uri', 'ng', 'soft', 'drinks', 'mayroon', 'kayo', '?']]

/mnt/c/ubuntu-bash/filipineu/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/mnt/c/ubuntu-bash/filipineu/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



['let', 'us', 'see', '.', 'hur', 'service', ',', 'beer', ',', 'but', 'here', ',', 'but', 'in', 'a', 'complator', '.']
[['nasaan', 'ang', 'tindahan', 'ng', 'pampalamig', '?']]
['where', 'is', 'the', 'rest', 'havior', '?']
[['pumupusta', 'ako', 'na', 'natutuwang', 'marinig', 'iyan', 'ni', 'fred', '.']]
['i', 'am', 'really', 'have', 'to', 'hear', 'that', 'driver', '.']
[['nais', 'kong', 'magmungkahi', 'ng', 'tagay', 'para', 'sa', 'ating', 'pagkakaibigan', 'at', 'kooperasyon', '.', 'chon', '!']]
['i', 'would', 'like', 'to', 'suggest', 'a', 'coopleation', '.', 'shool', '!']
[['magandang', 'gabi', ',', 'ginoo', '.']]
['good', 'evening', ',', 'sir', '.']
[['mas', 'mainit', 'dito', '.']]
['it', 'is', 'hot', 'it', '.']
[['magkano', 'ito', 'bawat', 'salita', '?']]
['what', 'is', 'the', 'cost', 'per', 'word', '?']
[['patis']]
['nice', 'been', 'are', '?']
[['nasaan', 'ang', 'labasan', '?']]
['where', 'is', 'the', 'exit', '?']
[['walang', 'pagong', ',', 'palaka', ',', 'igat', 'at', 'mga', 'katulad

/mnt/c/ubuntu-bash/filipineu/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



['looking', '.']
[['nasaan', 'ang', 'tindahan', 'ng', 'antigo', '?']]
['where', 'is', 'the', 'attable', 'store', '?']
[['tunog', 'interesante', '.']]
['sounding', '.']
[['narito', 'ang', 'pagkaing', 'inorder', 'mo', '.']]
['here', 'are', 'doine', '.']
[['100', 'kilometro', 'kada', 'oras', 'ang', 'limit', 'sa', 'tulin', '.']]
['the', 'speed', 'limit', 'is', '10', 'mileter', 'per', 'hour', '.']
[['hindi', 'ka', 'dapat', 'na', 'magdala', 'ng', 'mga', 'sariwang', 'prutas', 'papasok', 'sa', 'thailand', '.', 'kukumpiskahin', 'ko', 'sila', '.']]
['you', 'are', 'not', 'take', 'this', 'freshin', 'fruits', 'to', 'be', 'then', '.']
[['ano', 'ang', 'aking', 'gagawin', 'kung', 'gumawa', 'ng', 'presyon', 'sa', 'aking', 'tainga', 'ang', 'pagbabago', 'sa', 'altitude', '.']]
['what', 'shall', 'i', 'do', 'if', 'in', 'my', 'eares', 'change', 'in', 'my', 'each', '.']
[['kumanan', 'ka', 'sa', 'kalyeng', 'ito', '.']]
['tround', 'right', 'at', 'the', 'road', '.']
[['pakiusap', 'na', 'ipakita', 'ang', 'iyong

[['mayroon', 'kaming', 'dining', 'room', 'sa', 'ikatlong', 'palapag', '.']]
['we', 'have', 'a', 'dining', 'room', '1425', '.']
[['masakit', 'ang', 'aking', 'tiyan', '.']]
['i', 'have', 'a', 'stomachache', '.']
[['kukuha', 'ako', 'ng', 'margarita', '.']]
['i', 'will', 'have', 'a', 'margarior', '.']
[['hindi', 'humihinto', 'ang', 'mga', 'drayber', 'ng', 'bus', 'sa', 'bawat', 'hintuan', 'ng', 'bus', 'maliban', 'kung', 'may', 'mga', 'pasaherong', 'sasakay', 'o', 'bababa', '.']]
['the', 'bus', 'driver', 'does', 'not', 'stop', 'at', 'the', 'with', 'of', 'down', '.']
[['maaari', 'ba', 'akong', 'magkaroon', 'ng', 'mapa', 'ng', 'talibis', '?']]
['can', 'i', 'have', 'a', 'map', '?']
[['maaari', 'kang', 'diretsong', 'pumunta', 'doon', '.']]
['you', 'may', 'go', 'straight', 'there', '.']
[['lubhang', 'paumanhin', 'sa', 'nangyari', ',', 'ginoo', '.']]
['i', 'am', 'very', 'sorry', 'to', 'have', 'brent', ',', 'sir', '.']
[['maaari', 'mo', 'bang', 'dalhin', 'sa', 'amin', 'ang', 'bill', '?']]
['would',

[['magkano', 'ang', 'gusto', 'mong', 'palitan', '?']]
['how', 'much', 'do', 'you', 'want', 'to', 'change', '?']
[['sa', 'tingin', 'ko', ',', 'bagay', 'na', 'gawa', 'sa', 'lana', '.']]
['i', 'am', 'afraid', ',', 'it', 'is', 'mation', '.']
[['anong', 'uri', 'ng', 'alak', 'ang', 'inyong', 'gusto', '?']]
['which', 'kind', 'of', 'wine', 'do', 'you', 'prefer', '?']
[['nakintal', 'ako', 'sa', 'iyong', 'ingles', '.']]
['i', 'presjly', 'in', 'your', 'english', '.']
[['anong', 'uri', 'ng', 'permanente', 'ang', 'nais', 'niyo', '?']]
['what', 'kind', 'of', 'perfenter', 'would', 'you', 'like', '?']
[['mura', 'ang', 'tiket', 'na', 'ito', 'dahil', 'isang', 'itong', 'excursion', 'ticket', '.']]
['this', 'ticket', 'is', 'cheap', '.', 'it', 'is', 'a', 'cheap', '.']
[['darating', 'na', 'dito', 'sa', 'lalong', 'madaling', 'panahon', 'ang', 'susunod', 'na', 'subway', '?']]
['will', 'the', 'next', 'subway', 'be', 'coming', 'soon', '?']
[['kumanan', 'ka', 'sa', 'susunod', 'na', 'kalye', '.']]
['twent', 'of',

[['mabibigyan', 'mo', 'ba', 'ako', 'ng', '25', '%', 'na', 'diskuwentro', '?']]
['could', 'you', 'give', 'me', 'a', 'loss', 'discount', '?']
[['kadarating', 'lang', 'namin', '.']]
['we', 'ouge', 'just', 'lost', 'offect', '.']
[['itago', 'mo', 'ang', '5', 'baht', '.']]
['keep', 'the', '5', 'baht', '.']
[['magaang', 'toyo']]
['per', 'day', 'sauce']
[['nanakaw', 'ang', 'aking', 'salapi', '.']]
['my', 'money', 'was', 'stolen', '.']
[['ngunit', 'kailangan', 'mong', 'magbayad', 'para', 'sa', 'pagsasauli', 'ng', 'ibinayad', '.']]
['you', 'have', 'to', 'pay', 'for', 'refund', '.']
[['maaari', 'ko', 'bang', 'makita', 'ang', 'bill', '?']]
['can', 'i', 'see', 'the', 'bill', '?']
[['maraming', 'salamat', 'talaga', '.']]
['thanks', 'really', '.']
[['saang', 'gate', 'ako', 'dapat', 'pumunta', 'para', 'sa', 'bus', '?']]
['ahe', 'would', 'i', 'go', 'to', 'for', 'the', 'bus', '?']
[['ginagawa', 'mo', 'ba', 'ito', 'madalas', '?']]
['do', 'you', 'do', 'this', 'often', '?']
[['maiba', 'ako', ',', 'maaari',

[['isa', 'ka', 'bang', 'magaling', 'na', 'manlalaro', '?']]
['are', 'you', 'a', 'good', 'player', '?']
[['labis', 'ito', 'sa', 'kaya', 'kong', 'kainin', '.']]
['it', 'is', 'too', 'exciting', '.']
[['ano', 'ang', 'country', 'code', 'para', 'sa', 'thailand', '?']]
['what', 'is', 'the', 'country', 'code', 'for', 'thailand', '?']
[['pakiusap', 'dalhin', 'mo', 'sa', 'akin', 'ang', 'bill', '.']]
['please', 'bring', 'me', 'the', 'bill', '.']
[['kailangan', 'ko', 'ng', 'isang', 'bagong', 'customs', 'form', '.']]
['i', 'need', 'a', 'new', 'customs', 'form', '.']
[['magbubukas', 'ito', 'sa', 'oras', 'na', 'magsisimula', 'ang', 'tren', '.']]
['it', 'does', 'those', 'time', 'in', 'time', '.']
[['pakiusap', 'po', ',', 'saan', 'ang', 'susunod', 'na', 'tren', 'patungong', 'chiang', 'mai', '?']]
['where', 'is', 'the', 'next', 'train', 'to', 'chiang', 'mai', '?']
[['mayroon', 'bang', 'hair', 'stylist', 'sa', 'hotel', '?']]
['is', 'there', 'a', 'hishing', 'room', 'in', 'the', 'hotel', '?']
[['anong', 'u

In [8]:
bp = brevity_penalty(ref_lengths, hyp_lengths)

In [9]:
p_n = [Fraction(p_numerators[i], p_denominators[i], _normalize=False)
    for i, _ in enumerate(weights, start=1)]

In [10]:
smoothing_function = SmoothingFunction().method0

p_n = smoothing_function(p_n, references=references, hypothesis=hypothesis,
                             hyp_len=hyp_len, emulate_multibleu=False)

s = (w * math.log(p_i) for i, (w, p_i) in enumerate(zip(weights, p_n)))

print(math.exp(math.fsum(s)))

0.6774323093167594


/mnt/c/ubuntu-bash/filipineu/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [11]:
from nltk.translate.bleu_score import corpus_bleu

corpus_bleu(references, hypotheses)

/mnt/c/ubuntu-bash/filipineu/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6774323093167594